# arXiv heatmaps
We explore here the SARIMA model.

In [ ]:
import pandas as pd

full_df = pd.read_parquet("../../data/arxiv-totals.parquet")

df = (
    pd.DataFrame(full_df[list(full_df.filter(regex="math."))].sum(axis=1))
    .reset_index(inplace=False)
    .rename(columns={0: "math"})
)

df_train = df[
    (df.date >= pd.Timestamp(2001, 1, 1)) & (df.date <= pd.Timestamp(2025, 3, 14))
]
df_test = df[df.date >= pd.Timestamp(2025, 3, 17)]

We will try a SARIMA(0,0,0)(1,1,1) model with season m=5.  This is for computational simplicity but also because we believe that the only relevant effect is week-to-week (i.e. only seasonality).

Make the training splits.

In [19]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(n_splits=5, test_size=15, max_train_size=150)

splits = list(ts_cv.split(df_train))

In [ ]:
import statsmodels.tsa.api as sm
from sklearn.metrics import root_mean_squared_error
import numpy as np

rmses = np.zeros(5)

for i, (train_index, test_index) in enumerate(splits):
    df_tt = df_train.iloc[train_index, :]
    df_holdout = df_train.iloc[test_index, :]
    model = sm.ARIMA(
        df_train["math"], order=(0, 0, 0), seasonal_order=(1, 1, 1, 5)
    ).fit()
    preds = model.forecast(15)
    rmses[i] = root_mean_squared_error(df_holdout["math"], preds)

/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarn

The resulting root mean squared errors are

In [52]:
print("SARIMA rmses:", rmses)
print("  - mean = ", rmses.mean())

SARIMA rmses: [35.8039208  82.38198611 25.34369501 26.34654636 26.49175042]
  - mean =  39.273579740680944


This is better than any other model (compare with `short-term-math.ipynb`).

## Categorical SARIMA
We now do the same but split by categories (i.e. without aggregating `math`).

In [53]:
df = pd.read_parquet("../../data/arxiv-totals.parquet").reset_index(inplace=False)

# make train/test splits
df_train = df[
    (df.date >= pd.Timestamp(2001, 1, 1)) & (df.date <= pd.Timestamp(2025, 3, 14))
]
df_test = df[df.date >= pd.Timestamp(2025, 3, 17)]

# make 5-fold validations
ts_cv = TimeSeriesSplit(n_splits=5, test_size=15, max_train_size=150)
splits = list(ts_cv.split(df_train))

In [ ]:
import json

with open("../../data/arxiv-categories.json", "r") as f:
    arxiv_categories_descriptions = json.load(f)

categories = sorted(
    [cat["tag"] for cat in arxiv_categories_descriptions]
    # We exclude ["q-bio", "cond-mat", "astro-ph"] because they disappeared before our sample starting date.
)
results_dict = {}

for j, category in enumerate(categories):
    print("\n", category, "(" + str(j + 1) + "/" + str(len(categories)) + ")")
    rmses = np.zeros(5)
    for i, (train_index, test_index) in enumerate(splits):
        df_tt = df_train.iloc[train_index, :]
        df_holdout = df_train.iloc[test_index, :]
        model = sm.ARIMA(
            df_train[category], order=(0, 0, 0), seasonal_order=(1, 1, 1, 5)
        ).fit()
        preds = model.forecast(15)
        rmses[i] = root_mean_squared_error(df_holdout[category], preds)
    results_dict[category] = rmses.mean() / df_train[category].mean()


 astro-ph.CO (1/155)


/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarn


 astro-ph.EP (2/155)


/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarn


 astro-ph.GA (3/155)


...

/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarn


 stat.OT (154/155)


/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarn


 stat.TH (155/155)


/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/home/roberto/anaconda3/envs/arxiv-heatmaps/lib/python3.13/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarn

In [ ]:
sarima_results = pd.DataFrame(results_dict, index=[0]).T.rename(columns={0: "SARIMA"})
previous_results = pd.read_csv("short-term-linear-scaled-results.csv")

model_comparison = (
    previous_results.rename(columns={"Unnamed: 0": 0}).set_index(0).join(sarima_results)
)

In [85]:
model_comparison

,dummy,t_reg,day_reg,mday_reg,tday_reg,gbrt,prophet,prophet_full,SARIMA
0,,,,,,,,,
astro-ph.CO,0.510593,0.511111,0.463992,0.580969,0.465267,1.663033,1.483159,0.432457,0.428468
astro-ph.EP,0.857071,0.878727,0.837521,0.855076,0.858652,1.903437,1.768274,0.783602,0.783180
astro-ph.GA,0.779733,0.763716,0.750331,0.870479,0.734992,2.302193,2.225845,0.711223,0.726467
astro-ph.HE,0.730675,0.728761,0.688845,0.793961,0.686632,2.140697,2.149568,0.652805,0.663298
astro-ph.IM,0.978891,1.035259,0.929418,0.944577,0.982128,2.360880,2.223218,0.881011,0.874806
...,...,...,...,...,...,...,...,...,...
stat.CO,1.350181,1.343354,1.318394,1.432783,1.307019,2.903535,2.868624,1.298259,1.247398
stat.ME,1.266954,1.276913,0.989934,1.075921,1.008210,3.234106,3.345036,1.109691,0.939753
stat.ML,1.041047,1.102858,0.774462,0.837341,0.857019,2.274697,2.615044,1.397176,0.752098


In [86]:
model_comparison.mean()

dummy           1.475197
t_reg           1.511546
day_reg         1.378326
mday_reg        1.417188
tday_reg        1.417373
gbrt            2.570310
prophet         2.529926
prophet_full    1.409959
SARIMA          1.337562
dtype: float64

SARIMA seems to be faring better than any other model.

In [92]:
sarima_is_best = []
margin = []
for index, content in model_comparison.iterrows():
    if content.min() == content["SARIMA"]:
        sarima_is_best.append(index)
        margin.append(content["day_reg"] - content["SARIMA"])

print(
    "SARIMA is the best model for",
    len(sarima_is_best) / len(categories) * 100,
    "% of the categories.",
)
print()
print("The categories are the following:")
for i in range(len(sarima_is_best)):
    print("    -", sarima_is_best[i])
    print("        day_rmse - sarima_rmse =", margin[i])

print()
print("max improvement over day_reg =", np.array(margin).max())
print("mean improvement over day_reg =", np.array(margin).mean())

print()
print("SARIMA is not the best model for the following categories:")
for cat in sorted(set(categories).difference(set(sarima_is_best))):
    print("    -", cat)

SARIMA is the best model for 60.64516129032258 % of the categories.

The categories are the following:
    - astro-ph.CO
        day_rmse - sarima_rmse = 0.03552375387041634
    - astro-ph.EP
        day_rmse - sarima_rmse = 0.05434088270561732
    - astro-ph.IM
        day_rmse - sarima_rmse = 0.05461182291684519
    - astro-ph.SR
        day_rmse - sarima_rmse = 0.028224778240000803
    - cond-mat.dis-nn
        day_rmse - sarima_rmse = 0.02246220030724755
    - cond-mat.mes-hall
        day_rmse - sarima_rmse = 0.00614718661632685
    - cond-mat.mtrl-sci
        day_rmse - sarima_rmse = 0.030090829511245976
    - cond-mat.other
        day_rmse - sarima_rmse = 0.032279878073369295
    - cond-mat.soft
        day_rmse - sarima_rmse = 0.018462448054026193
    - cs.AR
        day_rmse - sarima_rmse = 0.09649303343721138
    - cs.CE
        day_rmse - sarima_rmse = 0.04231579150396847
    - cs.CR
        day_rmse - sarima_rmse = 0.012289740872922117
    - cs.CY
        day_rmse - sarima

In [93]:
model_comparison.to_csv("model-comparison.csv")